In [5]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel


def sample_simplex(n_samples=1000, total_volume=200):
    """
    Generate points uniformly on the 2-simplex scaled to total_volume.
    """
    points = []
    while len(points) < n_samples:
        u, v = np.random.rand(), np.random.rand()
        if u + v <= 1:
            w = 1 - u - v
            points.append([u, v, w])
    return np.array(points) * total_volume


def fit_gp(X, y, kernel=None, alpha=1e-6):
    """
    Fit a single-output GP for the distance.
    """
    if kernel is None:
        base = Matern(length_scale=50.0, nu=2.5)
        kernel = base + WhiteKernel(noise_level=1e-3)
    gp = GaussianProcessRegressor(kernel=kernel, alpha=alpha, normalize_y=True)
    gp.fit(X, y)
    return gp


def suggest_next(gp, total_volume=200, grid_size=1000):
    """
    Suggest the mixture predicted to minimize the distance.
    """
    grid = sample_simplex(grid_size, total_volume)
    mu, _ = gp.predict(grid, return_std=True)
    idx = np.argmin(mu)
    return grid[idx]


def interactive_loop():
    """
    Interactively suggest mixtures to find the unknown target color by minimizing measured distance.
    """
    X_data, y_data = [], []
    print("Welcome to the distance-based mixture explorer.")
    print("You only provide the Euclidean distance between the unknown target and the observed color.")
    print("Type 'quit' to exit at any prompt.\n")

    # Initial extreme point: test one pure liquid
    extremes = np.eye(3) * 200
    x_next = extremes[np.random.choice(3)]
    iteration = 0

    while True:
        iteration += 1
        print(f"Iteration {iteration}: Suggest mixture -> [A={x_next[0]:.1f}, B={x_next[1]:.1f}, C={x_next[2]:.1f}] µL")
        user = input("Enter observed distance (float) or 'quit': ").strip()
        if user.lower() == 'quit':
            print("Exiting. Goodbye!")
            break
        try:
            dist = float(user)
        except ValueError:
            print("Invalid input. Please enter a numeric distance.\n")
            continue

        # record data
        X_data.append(x_next)
        y_data.append(dist)

        # fit GP on distance
        X_arr = np.vstack(X_data)
        y_arr = np.array(y_data)
        gp = fit_gp(X_arr, y_arr)

        # suggest next mixture
        x_next = suggest_next(gp)
        print("")

if __name__ == '__main__':
    interactive_loop()


Welcome to the distance-based mixture explorer.
You only provide the Euclidean distance between the unknown target and the observed color.
Type 'quit' to exit at any prompt.

Iteration 1: Suggest mixture -> [A=0.0, B=200.0, C=0.0] µL


Enter observed distance (float) or 'quit':  117.2



Iteration 2: Suggest mixture -> [A=85.4, B=85.4, C=29.2] µL


Enter observed distance (float) or 'quit':  78.9



Iteration 3: Suggest mixture -> [A=86.3, B=87.1, C=26.6] µL


...


Iteration 11: Suggest mixture -> [A=82.5, B=86.8, C=30.7] µL


Enter observed distance (float) or 'quit':  107.2


C:\Users\1078620\AppData\Local\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(



Iteration 12: Suggest mixture -> [A=128.5, B=22.0, C=49.4] µL


KeyboardInterrupt: Interrupted by user

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel

MIN_VOL = 20.0    # minimum nonzero volume in µL
TOTAL_VOL = 200.0
GRID_SIZE = 2000  # candidate grid size


def sample_valid_simplex(n_samples, total_volume=TOTAL_VOL, min_vol=MIN_VOL):
    """
    Generate n_samples points on the simplex summing to total_volume,
    where each component is either 0 or >= min_vol.
    """
    pts = []
    while len(pts) < n_samples:
        u, v = np.random.rand(), np.random.rand()
        if u + v <= 1:
            w = 1 - u - v
            trip = np.array([u, v, w]) * total_volume
            # apply constraints: each vol == 0 (<= tiny) or >= min_vol
            if all((x < 1e-6 or x >= min_vol) for x in trip):
                pts.append(trip)
    return np.vstack(pts)


def fit_gp_distance(X, y, kernel=None, alpha=1e-6):
    """
    Fit a single-output GP for the distance objective.
    """
    if kernel is None:
        kernel = Matern(length_scale=50.0, nu=2.5) + WhiteKernel(noise_level=1e-3)
    gp = GaussianProcessRegressor(kernel=kernel, alpha=alpha, normalize_y=True)
    gp.fit(X, y)
    return gp


def suggest_next_lcb(gp_dist, total_volume=TOTAL_VOL, grid_size=GRID_SIZE, kappa=2.0):
    """
    Suggest the mixture minimizing the Lower Confidence Bound on distance:
        LCB(x) = mu(x) - kappa * sigma(x)
    under the volume constraints.
    """
    grid = sample_valid_simplex(grid_size)
    mu, std = gp_dist.predict(grid, return_std=True)
    lcb = mu - kappa * std
    return grid[np.argmin(lcb)]


def interactive_loop():
    """
    Interactive loop for distance-only optimization:
      - Suggest volumes
      - Prompt for observed distance
      - Update GP and suggest next
    Volumes are either 0 or >= MIN_VOL and sum to TOTAL_VOL.
    """
    X_data, y_dist = [], []
    print("=== Distance-Driven Mixture Optimizer ===")
    print("Enter the observed Euclidean distance each iteration; type 'quit' to exit.\n")

    # Initial extremes: pure A, B, or C
    extremes = np.eye(3) * TOTAL_VOL
    x_next = extremes[np.random.choice(3)]
    iteration = 0

    while True:
        iteration += 1
        a, b, c = [round(v) for v in x_next]
        print(f"Iteration {iteration}: Suggest volumes -> A={a}µL, B={b}µL, C={c}µL (sum={a+b+c})")

        user = input("Observed distance (or 'quit'): ").strip()
        if user.lower() == 'quit':
            print("Exiting. Goodbye!")
            break
        try:
            dist = float(user)
        except ValueError:
            print("Invalid input. Please enter a numeric distance.\n")
            iteration -= 1
            continue

        # Record data
        X_data.append([a, b, c])
        y_dist.append(dist)
        X_arr = np.vstack(X_data)
        y_arr = np.array(y_dist)

        # Fit GP on distance
        gp_dist = fit_gp_distance(X_arr, y_arr)

        # Suggest next mixture
        x_next = suggest_next_lcb(gp_dist)
        print("")

if __name__ == '__main__':
    interactive_loop()

=== Distance-Driven Mixture Optimizer ===
Enter the observed Euclidean distance each iteration; type 'quit' to exit.

Iteration 1: Suggest volumes -> A=200µL, B=0µL, C=0µL (sum=200)


Observed distance (or 'quit'):  67.5



Iteration 2: Suggest volumes -> A=20µL, B=159µL, C=21µL (sum=200)


Observed distance (or 'quit'):  90.8



Iteration 3: Suggest volumes -> A=159µL, B=21µL, C=20µL (sum=200)


...


Iteration 9: Suggest volumes -> A=91µL, B=35µL, C=74µL (sum=200)


Observed distance (or 'quit'):  21.4



Iteration 10: Suggest volumes -> A=105µL, B=46µL, C=49µL (sum=200)


Observed distance (or 'quit'):  9.5



Iteration 11: Suggest volumes -> A=104µL, B=70µL, C=26µL (sum=200)
